### 라이브러리 로딩

In [2]:
# 라이브러리 로딩
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from model_class import ResumeScreeningModel 

In [3]:
# 데이터 로딩
df = pd.read_csv("./src/ProcessedResumeDataSet.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 962 entries, 0 to 961
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Category        962 non-null    object
 1   Resume          962 non-null    object
 2   cleaned_resume  962 non-null    object
 3   skills          962 non-null    object
 4   education       962 non-null    object
 5   experience      962 non-null    object
dtypes: object(6)
memory usage: 45.2+ KB


,Category,Resume,cleaned_resume,skills,education,experience
0,Data Science,Skills * Programming Languages: Python (pandas...,skill programming language python panda numpy ...,"['machine learning', 'numpy', 'deep learning',...",[],['Education Details \r\n\r\nData Science Assur...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education detail may 2013 may 2017 b.e uitrgpv...,"['machine learning', 'python']",[],['Education Details \r\nMay 2013 to May 2017 B...
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",area interest deep learning control system des...,"['machine learning', 'deep learning', 'python']",['B.Tech'],"['Areas of Interest Deep Learning, Control Sys..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skill python sap hana tableau sap hana sql sap...,"['machine learning', 'deep learning', 'python']",['Bachelor'],['Skills â\x80¢ R â\x80¢ Python â\x80¢ SAP HAN...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education detail mca ymcaust faridabad haryana...,['python'],['MCA'],[]


In [4]:
# 필수 컬럼 확인
assert 'cleaned_resume' in df.columns, "cleaned_resume 컬럼이 없습니다."
assert 'Category' in df.columns, "Category 컬럼이 없습니다."

In [5]:
# 라벨 인코딩
df['label'] = df['Category'].astype('category').cat.codes
label_map = dict(enumerate(df['Category'].astype('category').cat.categories))

In [6]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=3000)
X = vectorizer.fit_transform(df['cleaned_resume'])

In [6]:
# 모델 클래스 인스턴스 생성 및 학습
modeler = ResumeScreeningModel()
best = modeler.train_with_best_scaler(data=df, features=X, target_column='label', scoring='f1_weighted')


===== Scaler: Standard =====
-*** LogisticRegression with Standard ***-
훈련 F1: 1.0000, 검증 F1: 1.0000, 테스트 F1: 0.9949, 과대적합 차이: 0.0000
훈련 F1 1.0으로 과대적합, 사용 불가능한 모델입니다.

LogisticRegression - F1: 0.9949, Usable: N, Status: 과대적합
-*** RandomForest with Standard ***-
훈련 F1: 1.0000, 검증 F1: 1.0000, 테스트 F1: 0.9949, 과대적합 차이: 0.0000
훈련 F1 1.0으로 과대적합, 사용 불가능한 모델입니다.

RandomForest - F1: 0.9949, Usable: N, Status: 과대적합
-*** GradientBoosting with Standard ***-
훈련 F1: 1.0000, 검증 F1: 1.0000, 테스트 F1: 1.0000, 과대적합 차이: 0.0000
훈련 F1 1.0으로 과대적합, 사용 불가능한 모델입니다.

GradientBoosting - F1: 1.0000, Usable: N, Status: 과대적합
-*** SVM with Standard ***-
훈련 F1: 1.0000, 검증 F1: 1.0000, 테스트 F1: 0.9949, 과대적합 차이: 0.0000
훈련 F1 1.0으로 과대적합, 사용 불가능한 모델입니다.

SVM - F1: 0.9949, Usable: N, Status: 과대적합
-*** KNN with Standard ***-
훈련 F1: 0.8971, 검증 F1: 0.8090, 테스트 F1: 0.7757, 과대적합 차이: 0.0881
사용 가능한 모델입니다 (일반화).

KNN - F1: 0.7757, Usable: Y, Status: 일반화
-*** DecisionTree with Standard ***-
훈련 F1: 1.0000, 검증 F1: 1.0000, 테스트 F1: 1.000

In [7]:
# 결과 출력
print(f"\n최종 선택된 모델: {best['model']}")
print(f"사용된 스케일러: {best['scaler']}")
print(f"F1 Score: {best['metrics']['f1']:.4f}")
print(f"하이퍼파라미터: {best['metrics']['params']}")


최종 선택된 모델: KNN
사용된 스케일러: None
F1 Score: 0.9348
하이퍼파라미터: {}


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier

# KNN Only Training and Evaluation Function
def train_knn_only(data, features, target_column='label'):
    print(f"\n===== KNN Only | Scaler: None =====")

    # 데이터 분할 (scaler 없이)
    X = features
    y = data[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, stratify=y_train, random_state=42)

    # KNN 모델
    model = KNeighborsClassifier(n_neighbors=5)

    # 훈련
    model.fit(X_train, y_train)

    # 예측
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)

    # 점수
    train_f1 = f1_score(y_train, y_train_pred, average='weighted')
    val_f1 = f1_score(y_val, y_val_pred, average='weighted')
    test_f1 = f1_score(y_test, y_test_pred, average='weighted')
    overfit_diff = train_f1 - val_f1

    # 결과 출력
    print(f"훈련 F1: {train_f1:.4f}, 검증 F1: {val_f1:.4f}, 테스트 F1: {test_f1:.4f}, 과대적합 차이: {overfit_diff:.4f}")
    if train_f1 < 1.0 and 0.0 <= overfit_diff <= 0.1:
        print("사용 가능한 모델입니다 (일반화).\n")
    else:
        print("사용 불가능한 모델입니다.\n")

    # 결과 반환
    return {
        'train_f1': train_f1,
        'val_f1': val_f1,
        'test_f1': test_f1,
        'overfit_diff': overfit_diff,
        'usable': 'Y' if train_f1 < 1.0 and overfit_diff <= 0.1 else 'N',
        'overfit_status': '일반화' if train_f1 < 1.0 and overfit_diff <= 0.1 else '과대적합',
        'model': model
    }



In [8]:
# 모델 훈련
result = train_knn_only(data=df, features=X, target_column='label')

# 모델 저장
import joblib
joblib.dump(result['model'], './model/knn_model.pkl')
print("KNN 모델 저장 완료: knn_model.pkl")



===== KNN Only | Scaler: None =====
훈련 F1: 0.9734, 검증 F1: 0.9389, 테스트 F1: 0.9348, 과대적합 차이: 0.0345
사용 가능한 모델입니다 (일반화).

KNN 모델 저장 완료: knn_model.pkl


In [9]:
# 모델 저장
import joblib
joblib.dump(result['model'], './model/knn_model.pkl')
print("KNN 모델 저장 완료: knn_model.pkl")

# JSON으로 저장 가능한 항목만 추출
json_result = {k: v for k, v in result.items() if k != 'model'}

# 메트릭 결과 저장
import json
with open('./model/knn_metrics.json', 'w') as f:
    json.dump(json_result, f, indent=4)
print("KNN 평가 결과 저장 완료: knn_metrics.json")


KNN 모델 저장 완료: knn_model.pkl
KNN 평가 결과 저장 완료: knn_metrics.json


In [13]:
# 모델 로드
knn_model = joblib.load('./model/knn_model.pkl')

# 예측
sample_idx = 300
sample_resume = vectorizer.transform([df['cleaned_resume'][sample_idx]])
predicted_label = knn_model.predict(sample_resume)[0]
actual_label = df['Category'][sample_idx]

print(f"\n샘플 인덱스: {sample_idx}")
print("실제 카테고리:", actual_label)
print("예측된 카테고리:", label_map[predicted_label])


샘플 인덱스: 300
실제 카테고리: Civil Engineer
예측된 카테고리: Civil Engineer
